## This script loads, filters and merges data on NYC population and income

The income data is from the American Community Survey (ACS) 5 year file (2018-2022). Downloaded from https://www.nyc.gov/content/planning/pages/resources/datasets/american-community-survey on 17.07.25

Population data from U.S. Census Bureau, 2010 and 2020 Census Redistricting Data (Public Law 94-171) Summary Files. Downloaded from https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/census2020/nyc_decennialcensusdata_2010_2020_change.xlsx on 17.07.25 

GeoJSON data on NY NTAs downloaded on 21.07.25 from: 
https://data.cityofnewyork.us/City-Government/2020-Neighborhood-Tabulation-Areas-NTAs-/9nt8-h7nd/about_data 

In [76]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
import geopandas as gpd

In [ ]:
# Loading population data. Pop_20 column is population of a given NTA in 2020
census = pd.read_excel(
    r'C:/Data/Citibike_NY_2022/nyc_decennialcensusdata_2010_2020_change.xlsx',
    sheet_name='2010, 2020, and Change',
    header = 3,
    usecols=['GeoType', "Borough", "GeoID", "Name", "NTA Type", "Pop_20"]
    )

In [4]:
census.head()

,GeoType,Borough,GeoID,Name,NTA Type,Pop_20
0,NYC,New York City,0,NYC,NaN,8804190
1,Boro,Manhattan,1,Manhattan,NaN,1694251
2,Boro,Bronx,2,Bronx,NaN,1472654
3,Boro,Brooklyn,3,Brooklyn,NaN,2736074
4,Boro,Queens,4,Queens,NaN,2405464


In [ ]:
# Limiting rows to NTAs (excluding other aggregation types like borough)
NTA_census = census[census['NTA Type'].notna()]

In [6]:
NTA_census.shape

(262, 6)

In [7]:
NTA_census.head()

,GeoType,Borough,GeoID,Name,NTA Type,Pop_20
128,NTA2020,Brooklyn,BK0101,Greenpoint,0.0,38980
129,NTA2020,Brooklyn,BK0102,Williamsburg,0.0,64444
130,NTA2020,Brooklyn,BK0103,South Williamsburg,0.0,47703
131,NTA2020,Brooklyn,BK0104,East Williamsburg,0.0,52998
132,NTA2020,Brooklyn,BK0201,Brooklyn Heights,0.0,25092


In [17]:
NTA_census['Borough'].value_counts()

Borough
Queens           82
Brooklyn         69
Bronx            50
Manhattan        38
Staten Island    23
Name: count, dtype: int64

In [35]:
NTA_census.isna().sum()

GeoType     0
Borough     0
GeoID       0
Name        0
NTA Type    0
Pop_20      0
dtype: int64

## Loading Income Data

In [21]:
# MdHHIncE is the estimate of median household income
income = pd.read_excel(r'C:/Data/Citibike_NY_2022/Econ_1822_NTA.xlsx',
                       sheet_name='EconData',
                       header=0,
                       usecols=["GeoType", "NTAType", "GeogName", "GeoID", "Borough", 'MdHHIncE'])

In [22]:
income.head()

,GeoType,NTAType,GeogName,GeoID,Borough,MdHHIncE
0,NTA2020,0,Greenpoint,BK0101,Brooklyn,125469.0
1,NTA2020,0,Williamsburg,BK0102,Brooklyn,129838.0
2,NTA2020,0,South Williamsburg,BK0103,Brooklyn,36951.0
3,NTA2020,0,East Williamsburg,BK0104,Brooklyn,71107.0
4,NTA2020,0,Brooklyn Heights,BK0201,Brooklyn,179877.0


In [23]:
income.shape

(262, 6)

In [24]:
income['Borough'].value_counts()

Borough
Queens           83
Brooklyn         69
Bronx            49
Manhattan        38
Staten Island    23
Name: count, dtype: int64

One more queens and one less Bronx than the population df

In [34]:
income.isna().sum()

GeoType      0
NTAType      0
GeogName     0
GeoID        0
Borough      0
MdHHIncE    51
dtype: int64

In [36]:
inc_missings = income[income['MdHHIncE'].isna()].copy()

In [39]:
inc_missings['NTAType'].value_counts()

NTAType
9    32
7    11
6     5
8     2
5     1
Name: count, dtype: int64

Can see from NTA codes (5-9), that these are all non residential codes, so OK to drop

In [40]:
income = income.dropna(subset=['MdHHIncE'])

In [41]:
income.isna().sum()

GeoType     0
NTAType     0
GeogName    0
GeoID       0
Borough     0
MdHHIncE    0
dtype: int64

## Merging into one df

In [51]:
pop_inc = NTA_census.merge(income[['GeoID', 'MdHHIncE']],
                           how='left',
                           on='GeoID',
                           indicator=True
                           )

In [52]:
pop_inc.head()

,GeoType,Borough,GeoID,Name,NTA Type,Pop_20,MdHHIncE,_merge
0,NTA2020,Brooklyn,BK0101,Greenpoint,0.0,38980,125469.0,both
1,NTA2020,Brooklyn,BK0102,Williamsburg,0.0,64444,129838.0,both
2,NTA2020,Brooklyn,BK0103,South Williamsburg,0.0,47703,36951.0,both
3,NTA2020,Brooklyn,BK0104,East Williamsburg,0.0,52998,71107.0,both
4,NTA2020,Brooklyn,BK0201,Brooklyn Heights,0.0,25092,179877.0,both


In [54]:
pop_inc['_merge'].value_counts()

_merge
both          211
left_only      51
right_only      0
Name: count, dtype: int64

In [55]:
pop_inc['GeoType'].value_counts()

GeoType
NTA2020    262
Name: count, dtype: int64

In [57]:
pop_inc['NTA Type'].value_counts()

NTA Type
0.0    197
9.0     40
7.0     14
6.0      8
8.0      2
5.0      1
Name: count, dtype: int64

55 non residential NTAs

In [64]:
# Looking into which NTAs are non residential yet have income data, to see if any issues with dropping them
non_residential = pop_inc[(pop_inc['NTA Type'] >= 5) & (pop_inc['MdHHIncE'].notna())].copy()

In [65]:
non_residential.shape

(14, 8)

In [67]:
non_residential.head(14)

,GeoType,Borough,GeoID,Name,NTA Type,Pop_20,MdHHIncE,_merge
33,NTA2020,Brooklyn,BK1061,Fort Hamilton,6.0,775,129317.0,both
58,NTA2020,Brooklyn,BK1771,Holy Cross Cemetery,7.0,32,174998.0,both
66,NTA2020,Brooklyn,BK5691,Barren Island-Floyd Bennett Field,9.0,21,137498.0,both
115,NTA2020,Bronx,BX2691,Van Cortlandt Park,9.0,122,84374.0,both
116,NTA2020,Bronx,BX2791,Bronx Park,9.0,69,67498.0,both
117,NTA2020,Bronx,BX2891,Pelham Bay Park,9.0,28,17915.0,both
162,NTA2020,Queens,QN0161,Sunnyside Yards (North),6.0,27,174998.0,both
179,NTA2020,Queens,QN0571,Mount Olivet & All Faiths Cemeteries,7.0,3,87498.0,both
182,NTA2020,Queens,QN0574,Highland Park-Cypress Hills Cemeteries (North),7.0,20,156249.0,both
238,NTA2020,Queens,QN8492,Jacob Riis Park-Fort Tilden-Breezy Point Tip,9.0,24,200000.0,both


Going to drop all non residential NTAs. These with income data are mostly all very small populations, with exceptions of Fort Wadsworth and Fort Hamilton. I checked both in the original Excel and the margins of error are also pretty large for the income variable (Hamilton Estimate: 129,317, Margin of error: 38,240; Wadsworth Estimate: 80,654, Margin of Error: 49,796).

In [ ]:
# Keeping all NTAs of type 0 (residential), that have income data
pop_inc_res = pop_inc[(pop_inc['NTA Type'] == 0) & (pop_inc['MdHHIncE'].notna())].copy()
pop_inc_res.shape

(197, 8)

In [70]:
pop_inc_res.head()

,GeoType,Borough,GeoID,Name,NTA Type,Pop_20,MdHHIncE,_merge
0,NTA2020,Brooklyn,BK0101,Greenpoint,0.0,38980,125469.0,both
1,NTA2020,Brooklyn,BK0102,Williamsburg,0.0,64444,129838.0,both
2,NTA2020,Brooklyn,BK0103,South Williamsburg,0.0,47703,36951.0,both
3,NTA2020,Brooklyn,BK0104,East Williamsburg,0.0,52998,71107.0,both
4,NTA2020,Brooklyn,BK0201,Brooklyn Heights,0.0,25092,179877.0,both


In [73]:
df_for_export = pop_inc_res[['GeoID', 'Name', 'Pop_20', 'MdHHIncE']].copy()
df_for_export.head()

,GeoID,Name,Pop_20,MdHHIncE
0,BK0101,Greenpoint,38980,125469.0
1,BK0102,Williamsburg,64444,129838.0
2,BK0103,South Williamsburg,47703,36951.0
3,BK0104,East Williamsburg,52998,71107.0
4,BK0201,Brooklyn Heights,25092,179877.0


In [74]:
df_for_export.rename(columns={'MdHHIncE': 'median_hh_income',
                              'Pop_20': 'population',
                              'Name': 'Borough'},
                              inplace=True)

In [75]:
# Saving as csv
df_for_export.to_csv('C:/Data/Citibike_NY_2022/merged/population_income.csv',
                     index=False)

Merging income and population data to GeoJson file of NTA shapes

In [80]:
# GeoJSON data on NY NTAs downloaded on 21.07.25 from: 
# https://data.cityofnewyork.us/City-Government/2020-Neighborhood-Tabulation-Areas-NTAs-/9nt8-h7nd/about_data
nta_geo = gpd.read_file("C:/Data/Citibike_NY_2022/2020 Neighborhood Tabulation Areas (NTAs)_20250721.geojson")

In [81]:
nta_geo.head()

,shape_area,ntaname,cdtaname,shape_leng,boroname,ntatype,nta2020,borocode,countyfips,ntaabbrev,cdta2020,geometry
0,35321809.1041,Greenpoint,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28919.5608108,Brooklyn,0,BK0101,3,047,Grnpt,BK01,"MULTIPOLYGON (((-73.93213 40.72816, -73.93238 ..."
1,28852852.7038,Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28134.0823238,Brooklyn,0,BK0102,3,047,Wllmsbrg,BK01,"MULTIPOLYGON (((-73.95814 40.7244, -73.95772 4..."
2,15208960.7339,South Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18250.2805432,Brooklyn,0,BK0103,3,047,SWllmsbrg,BK01,"MULTIPOLYGON (((-73.95024 40.70547, -73.94984 ..."
3,52267407.9898,East Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43184.7989883,Brooklyn,0,BK0104,3,047,EWllmsbrg,BK01,"MULTIPOLYGON (((-73.92406 40.71411, -73.92404 ..."
4,9982321.59069,Brooklyn Heights,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14312.5049745,Brooklyn,0,BK0201,3,047,BkHts,BK02,"MULTIPOLYGON (((-73.99236 40.68969, -73.99436 ..."


In [82]:
nta_pop_inc = nta_geo.merge(df_for_export[['GeoID', 'median_hh_income', 'population']],
                            how = 'left',
                            left_on = 'nta2020',
                            right_on = 'GeoID')

In [83]:
nta_pop_inc.head()

,shape_area,ntaname,cdtaname,shape_leng,boroname,ntatype,nta2020,borocode,countyfips,ntaabbrev,cdta2020,geometry,GeoID,median_hh_income,population
0,35321809.1041,Greenpoint,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28919.5608108,Brooklyn,0,BK0101,3,047,Grnpt,BK01,"MULTIPOLYGON (((-73.93213 40.72816, -73.93238 ...",BK0101,125469.0,38980.0
1,28852852.7038,Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28134.0823238,Brooklyn,0,BK0102,3,047,Wllmsbrg,BK01,"MULTIPOLYGON (((-73.95814 40.7244, -73.95772 4...",BK0102,129838.0,64444.0
2,15208960.7339,South Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18250.2805432,Brooklyn,0,BK0103,3,047,SWllmsbrg,BK01,"MULTIPOLYGON (((-73.95024 40.70547, -73.94984 ...",BK0103,36951.0,47703.0
3,52267407.9898,East Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43184.7989883,Brooklyn,0,BK0104,3,047,EWllmsbrg,BK01,"MULTIPOLYGON (((-73.92406 40.71411, -73.92404 ...",BK0104,71107.0,52998.0
4,9982321.59069,Brooklyn Heights,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14312.5049745,Brooklyn,0,BK0201,3,047,BkHts,BK02,"MULTIPOLYGON (((-73.99236 40.68969, -73.99436 ...",BK0201,179877.0,25092.0


In [ ]:
# Checking CRS of data
print(nta_pop_inc.crs)

EPSG:4326


In [86]:
# Reproject to a projected CRS (EPSG:2263 is NY State Plane, in feet)
nta_pop_inc = nta_pop_inc.to_crs(epsg=2263)

# Calculate area in square kilometers (note: 1 km² = 10.7639 * 1e6 ft²)
nta_pop_inc["area_km2"] = nta_pop_inc.geometry.area / (10.7639 * 1e6)

# Calculate population density (people per km²)
nta_pop_inc["pop_density"] = nta_pop_inc["population"] / nta_pop_inc["area_km2"]

In [87]:
nta_pop_inc.head()

,shape_area,ntaname,cdtaname,shape_leng,boroname,ntatype,nta2020,borocode,countyfips,ntaabbrev,cdta2020,geometry,GeoID,median_hh_income,population,area_km2,pop_density
0,35321809.1041,Greenpoint,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28919.5608108,Brooklyn,0,BK0101,3,047,Grnpt,BK01,"MULTIPOLYGON (((1003059.997 204572.025, 100299...",BK0101,125469.0,38980.0,3.281501,11878.711959
1,28852852.7038,Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28134.0823238,Brooklyn,0,BK0102,3,047,Wllmsbrg,BK01,"MULTIPOLYGON (((995851.916 203199.332, 995969....",BK0102,129838.0,64444.0,2.680512,24041.677440
2,15208960.7339,South Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18250.2805432,Brooklyn,0,BK0103,3,047,SWllmsbrg,BK01,"MULTIPOLYGON (((998047.21 196303.325, 998157.9...",BK0103,36951.0,47703.0,1.412960,33761.039343
3,52267407.9898,East Williamsburg,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43184.7989883,Brooklyn,0,BK0104,3,047,EWllmsbrg,BK01,"MULTIPOLYGON (((1005302.497 199455.73, 1005307...",BK0104,71107.0,52998.0,4.855812,10914.343809
4,9982321.59069,Brooklyn Heights,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14312.5049745,Brooklyn,0,BK0201,3,047,BkHts,BK02,"MULTIPOLYGON (((986367.735 190549.239, 985813....",BK0201,179877.0,25092.0,0.927395,27056.433378


In [ ]:
nta_pop_inc = nta_pop_inc.dropna(subset=["pop_density"])    # dropping missings

In [90]:
# Saving file with nta polygons with population and income data
nta_pop_inc.to_file("C:/Data/Citibike_NY_2022/merged/nta_pop_inc.geojson", driver="GeoJSON")